# Sai Ram

https://www.kaggle.com/c/santander-customer-transaction-prediction

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import seaborn as sns
from sklearn import metrics, preprocessing
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


## Set the desired TensorFlow output level for this example

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)

## Set filepath + filename

Read CSV

In [4]:
import dask.dataframe as dd

df_train = dd.read_csv('test.csv')
# df_test = dd.read_csv('../input/test.csv')

# TextFileReader

In [6]:
df_train

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
npartitions=60,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,int64,float64,int64,float64,float64,float64,float64,int64,int64,float64,float64,int64,int64,object,object,object,int64,int64,object,object,object,float64,int64,float64,float64,float64,object,float64,float64,object,object,float64,float64,float64,float64,float64,object,float64,object,float64,int64,float64,object,float64,float64,float64,object,object,float64,object,object,object,int64,int64,object,object,object,float64,int64,object,int64,object,object,float64,float64,object,float64,float64,float64,int64,float64,int64,int64,int64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
chunksize = 100000
TextFileReader = pd.read_csv('train.csv',na_values=['NA','?'], engine='python', low_memory = True, sep=',', chunksize=chunksize, iterator=True)
df = pd.concat(TextFileReader, ignore_index=True)

In [4]:
df.set_index('ID_code',inplace = True)
col_names = df.columns
df.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [5]:
from sklearn import preprocessing

x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [6]:
X =  df.iloc[:, 1:].values 
y = df.iloc[:, 0].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [9]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=19, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [11]:
y_pred.sum()

0.0

In [12]:
unique, counts = np.unique(y_pred, return_counts=True)
unique

array([0.])

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[18016     0]
 [ 1984     0]]
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95     18016
         1.0       0.00      0.00      0.00      1984

   micro avg       0.90      0.90      0.90     20000
   macro avg       0.45      0.50      0.47     20000
weighted avg       0.81      0.90      0.85     20000

0.9008


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [155]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[18005    11]
 [ 1962    22]]
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95     18016
         1.0       0.67      0.01      0.02      1984

   micro avg       0.90      0.90      0.90     20000
   macro avg       0.78      0.51      0.48     20000
weighted avg       0.88      0.90      0.86     20000

0.90135


In [173]:
# Load actual test data

chunksize = 100000
TextFileReader = pd.read_csv('test.csv',na_values=['NA','?'], engine='python', low_memory = True, sep=',', chunksize=chunksize, iterator=True)
dfTest = pd.concat(TextFileReader, ignore_index=True)

In [174]:
dfTest.set_index('ID_code',inplace = True)
dfTest.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [175]:
test_col_names = dfTest.columns
test_index = dfTest.index

In [176]:
# Feature Scaling

sc = StandardScaler()
x_scaled = sc.fit_transform(dfTest.values)
# X_test = sc.transform(X_test)
dfTest = pd.DataFrame(x_scaled)
dfTest.columns = test_col_names

In [177]:
# x = dfTest.values # returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# dfTest = pd.DataFrame(x_scaled)
# dfTest.columns = test_col_names

In [178]:
unlabeledTest = dfTest.values

In [179]:
X_test

array([[-1.507, -0.569, -0.388, ...,  1.276,  0.903, -0.271],
       [-1.251,  0.328, -0.565, ..., -1.24 ,  0.745, -0.135],
       [ 0.492, -0.687,  1.007, ...,  0.746,  1.963, -0.744],
       ...,
       [ 0.62 , -0.261, -0.801, ..., -1.781,  1.14 ,  0.157],
       [ 0.438, -0.329,  0.587, ..., -0.142, -0.497, -2.313],
       [-1.066,  1.775, -0.784, ..., -2.119, -0.971, -1.75 ]])

In [180]:
# Run model on actual test data
pred = classifier.predict(unlabeledTest)

In [181]:
outFile = pd.DataFrame(pred, index=test_index)
# pred.sum()
outFile.columns = ['target']
outFile.to_csv('outFileRF.csv')

In [182]:
outFile.describe()

,target
count,200000.000000
mean,0.001190
std,0.034476
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [165]:
# #Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
# #Import the required packages
# #Import sklearn's feature selection algorithm
# from sklearn.feature_selection import SelectKBest
# #Import chi2 for performing chi square test 
# from sklearn.feature_selection import chi2

# #Split the data into input and target
# X =  X_train
# y = y_train

# #We will select the features using chi square
# test = SelectKBest(score_func=chi2, k=4)
# #Fit the function for ranking the features by score
# fit = test.fit(X, y)
# #Summarize scores 
# np.set_printoptions(precision=3) 
# #Apply the transformation on to dataset
# features = fit.transform(X)
# #Summarize selected features 
# features[0:5,:]

In [166]:
# #Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
# #Import the required packages
# #Import sklearn's feature selection algorithm
# from sklearn.feature_selection import SelectKBest
# #Import chi2 for performing chi square test 
# from sklearn.feature_selection import chi2
# #Import sklearn's PCA algorithm
# from sklearn.decomposition import PCA

# #Split the data into input and target
# X =  X_train
# y = y_train


# #Feature extraction
# pca = PCA(n_components=3) 
# fit = pca.fit(X)
# #Summarize components
# fit.explained_variance_ratio_
# fit.components_